# Task 2: Face Detection
## Implement YOLO v8 face detector and measure precision/recall metrics

### 1. Import Libraries

In [ ]:
import cv2
import numpy as np
from pathlib import Path
import sys
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd

# Add src to path
sys.path.insert(0, str(Path('..').resolve()))
from src.detection import FaceDetector

print("Libraries imported successfully!")

### 2. Setup Paths

In [ ]:
# Define paths
BASE_PATH = Path('..').resolve()
GALLERY_ALIGNED_PATH = BASE_PATH / 'data' / 'gallery_aligned'
VALIDATION_PATH = BASE_PATH / 'data' / 'validation'
MODEL_PATH = BASE_PATH / 'models' / 'yolov8n.pt'

print(f"Base path: {BASE_PATH}")
print(f"Validation path: {VALIDATION_PATH}")
print(f"Model path: {MODEL_PATH}")
print(f"Validation path exists: {VALIDATION_PATH.exists()}")

# List validation images
val_images = sorted(list(VALIDATION_PATH.glob('*.jpg')) + list(VALIDATION_PATH.glob('*.png')))
print(f"\nValidation images found: {len(val_images)}")
if val_images:
    print(f"Sample: {val_images[0]}")

### 3. Initialize Face Detector

In [ ]:
# Initialize detector
detector = FaceDetector(model_path=str(MODEL_PATH), confidence=0.5)
print("Face detector initialized!")
print(f"Model: {detector.model}")

### 4. Test Detection on Sample Images

In [ ]:
# Test on first validation image
test_image = val_images[0]
img = cv2.imread(str(test_image))
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

print(f"Testing image: {test_image.name}")
print(f"Image shape: {img.shape}")

# Detect faces
detections = detector.detect(img)
print(f"\nDetections found: {len(detections)}")

for i, det in enumerate(detections):
    print(f"\nDetection {i+1}:")
    print(f"  Bbox: {det['bbox']}")
    print(f"  Confidence: {det['confidence']:.4f}")
    print(f"  Area: {det['area']} pixels")

### 5. Visualize Detections

In [ ]:
# Draw detections
img_with_detections = img.copy()
for det in detections:
    x1, y1, x2, y2 = det['bbox']
    confidence = det['confidence']
    # Draw bounding box
    cv2.rectangle(img_with_detections, (x1, y1), (x2, y2), (0, 255, 0), 2)
    # Draw confidence score
    label = f"{confidence:.2f}"
    cv2.putText(img_with_detections, label, (x1, y1 - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

img_rgb_detected = cv2.cvtColor(img_with_detections, cv2.COLOR_BGR2RGB)

# Display
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img_rgb_detected)
plt.title(f'Detections (Found {len(detections)} faces)')
plt.axis('off')

plt.tight_layout()
plt.show()

print(f"✓ Detection visualization complete")

### 6. Run Detection on All Validation Images

In [ ]:
# Run detection on all validation images
all_detections = {}
detection_stats = []

print(f"Processing {len(val_images)} validation images...\n")

for val_img_path in tqdm(val_images, desc="Detecting faces"):
    img = cv2.imread(str(val_img_path))
    if img is None:
        continue
    
    detections = detector.detect(img)
    all_detections[val_img_path.name] = detections
    
    # Store stats
    detection_stats.append({
        'image': val_img_path.name,
        'num_detections': len(detections),
        'image_size': img.shape[:2],
        'max_confidence': max([d['confidence'] for d in detections]) if detections else 0
    })

print(f"\n✓ Detection complete for {len(all_detections)} images")

### 7. Detection Statistics

In [ ]:
# Create statistics dataframe
df_stats = pd.DataFrame(detection_stats)

print("Detection Statistics:")
print(f"\nTotal validation images: {len(df_stats)}")
print(f"Images with detections: {(df_stats['num_detections'] > 0).sum()}")
print(f"Images without detections: {(df_stats['num_detections'] == 0).sum()}")
print(f"\nDetections per image:")
print(f"  Min: {df_stats['num_detections'].min()}")
print(f"  Max: {df_stats['num_detections'].max()}")
print(f"  Mean: {df_stats['num_detections'].mean():.2f}")
print(f"\nConfidence scores:")
print(f"  Min: {df_stats['max_confidence'].min():.4f}")
print(f"  Max: {df_stats['max_confidence'].max():.4f}")
print(f"  Mean: {df_stats['max_confidence'].mean():.4f}")

print(f"\nDetailed results:")
print(df_stats.to_string())

### 8. Calculate Precision/Recall Metrics
Note: Since validation set has 1 face per image (from Task 1), expected ground truth = 1 face per image

In [ ]:
# For each validation image, expected ground truth = 1 face
# Calculate metrics assuming each image should have exactly 1 face detected

metrics_per_image = []

for val_img_path in val_images:
    img = cv2.imread(str(val_img_path))
    if img is None:
        continue
    
    detections = all_detections.get(val_img_path.name, [])
    
    # Ground truth: 1 face per image (from Task 1 - already aligned)
    ground_truth = [[10, 10, img.shape[1]-10, img.shape[0]-10]]  # Approximate full image as GT
    
    # If detections exist, we have at least one prediction
    if detections:
        # True Positive if at least 1 detection
        tp = 1
        fp = max(0, len(detections) - 1)  # Extra detections are false positives
        fn = 0
    else:
        # No detection = False Negative
        tp = 0
        fp = 0
        fn = 1
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else (1.0 if fn == 0 else 0.0)
    recall = tp / (tp + fn) if (tp + fn) > 0 else 1.0
    
    metrics_per_image.append({
        'image': val_img_path.name,
        'num_detections': len(detections),
        'tp': tp,
        'fp': fp,
        'fn': fn,
        'precision': precision,
        'recall': recall
    })

df_metrics = pd.DataFrame(metrics_per_image)

# Overall metrics
total_tp = df_metrics['tp'].sum()
total_fp = df_metrics['fp'].sum()
total_fn = df_metrics['fn'].sum()

overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
f1_score = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

print("=" * 60)
print("DETECTION METRICS - OVERALL")
print("=" * 60)
print(f"Total Positive (TP): {total_tp}")
print(f"False Positive (FP): {total_fp}")
print(f"False Negative (FN): {total_fn}")
print(f"\nPrecision: {overall_precision:.4f}")
print(f"Recall: {overall_recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")
print("=" * 60)

### 9. Visualization of Metrics

In [ ]:
# Plot metrics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Detection count distribution
axes[0, 0].bar(range(len(df_metrics)), df_metrics['num_detections'])
axes[0, 0].set_xlabel('Image Index')
axes[0, 0].set_ylabel('Number of Detections')
axes[0, 0].set_title('Detections per Image')
axes[0, 0].grid(alpha=0.3)

# 2. Precision per image
axes[0, 1].bar(range(len(df_metrics)), df_metrics['precision'], color='green', alpha=0.7)
axes[0, 1].axhline(y=overall_precision, color='r', linestyle='--', label=f'Overall: {overall_precision:.2f}')
axes[0, 1].set_xlabel('Image Index')
axes[0, 1].set_ylabel('Precision')
axes[0, 1].set_title('Precision per Image')
axes[0, 1].set_ylim([0, 1.1])
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# 3. Recall per image
axes[1, 0].bar(range(len(df_metrics)), df_metrics['recall'], color='blue', alpha=0.7)
axes[1, 0].axhline(y=overall_recall, color='r', linestyle='--', label=f'Overall: {overall_recall:.2f}')
axes[1, 0].set_xlabel('Image Index')
axes[1, 0].set_ylabel('Recall')
axes[1, 0].set_title('Recall per Image')
axes[1, 0].set_ylim([0, 1.1])
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# 4. Overall metrics summary
axes[1, 1].axis('off')
metrics_text = f"""
FACE DETECTION METRICS
{'='*40}

Total Images: {len(df_metrics)}
Images with Detections: {(df_metrics['num_detections'] > 0).sum()}
Images without Detections: {(df_metrics['num_detections'] == 0).sum()}

True Positives (TP): {total_tp}
False Positives (FP): {total_fp}
False Negatives (FN): {total_fn}

Precision: {overall_precision:.4f}
Recall: {overall_recall:.4f}
F1-Score: {f1_score:.4f}
"""
axes[1, 1].text(0.1, 0.5, metrics_text, fontsize=12, family='monospace',
                verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("✓ Metrics visualization complete")

### 10. Summary

In [ ]:
print("\n" + "="*60)
print("TASK 2: FACE DETECTION - SUMMARY")
print("="*60)
print(f"\n✓ Model: YOLO v8 (nano)")
print(f"✓ Validation Set: {len(val_images)} images")
print(f"✓ Confidence Threshold: 0.5")
print(f"\n📊 METRICS:")
print(f"   Precision: {overall_precision:.4f} ({overall_precision*100:.2f}%)")
print(f"   Recall: {overall_recall:.4f} ({overall_recall*100:.2f}%)")
print(f"   F1-Score: {f1_score:.4f}")
print(f"\n✓ Faces correctly detected: {total_tp}/{total_tp + total_fn}")
print(f"✓ False positives: {total_fp}")
print("\n" + "="*60)